In [1]:
import random
import numpy as np
import pandas as pd
import time
import re
import json
import datetime
import json
import os
import matplotlib.pyplot as plt
import scipy
from scipy import stats

### Sentences

In [2]:
%%time
sdf = pd.read_excel('data/topic_model_data/fullSentDataFrame_8-3.xlsx',index_col=0)
print(len(sdf))
sdf = sdf.drop_duplicates(['sentText'])
print(len(sdf))
sdf.head()

56609
53349
CPU times: user 7.35 s, sys: 123 ms, total: 7.47 s
Wall time: 7.62 s


,index,newsOutlet,category,dateSeen,url,title,language,sourceCountry,sentText,sentIndexInText,sentTopicID
0,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,BRUSSELS (AP) — The European Union is consider...,0,-1
1,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,Acting on a request from Ukraine for help with...,1,-1
2,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,The topic will be discussed during a summit Tu...,2,-1
3,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,The results of the mission have yet to be anal...,3,5
4,0,apnews.com,western,2021-10-12 07:00:00,https://apnews.com/article/technology-business...,"EU , Ukraine to discuss military training and ...",English,United States,One official said the EU’s political and secur...,4,-1


In [3]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.tokenize import sent_tokenize
nltk.download('averaged_perceptron_tagger')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.vader import SentiText
nltk.download('vader_lexicon')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/paigelee/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [67]:
%%time
# add sentiment row to DF
sentiRecs = []
records = sdf.to_dict('records')
for rec in records:
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(rec['sentText'])
    artSenti = dict()
    for k in ['pos','neu','neg']:
        artSenti[k] = ss[k]
    sentiRecs.append(artSenti)

KeyboardInterrupt: 

In [68]:
sentOnlyDf = pd.DataFrame.from_records(sentiRecs)
df = pd.concat([sdf, sentOnlyDf],axis=1)
df['dateSeen'] = df['dateSeen'].apply(lambda d: pd.to_datetime(d))
df['sentiVal'] = df['pos']-df['neg']
df.sample(4)

,index,newsOutlet,category,dateSeen,url,title,language,sourceCountry,sentText,sentIndexInText,sentTopicID,pos,neu,neg,sentiVal
46458,2342.0,tass.com,russian,2022-01-01 08:00:00,https://tass.com/world/1383393,Israeli embassy in Kiev condemns nationalists ...,English,NaN,Any attempt to glorify those who supported Naz...,2.0,14.0,NaN,NaN,NaN,NaN
25270,1010.0,sputniknews.com,russian,2022-01-03 08:00:00,https://sputniknews.com/20220103/ramped-up-us-...,Ramped Up US Sanctions Sending Cuba Closer to ...,English,Russia,The People’s Republic of China (PRC) is alread...,23.0,-1.0,0.130,0.870,0.00,0.130
25388,1013.0,sputniknews.com,russian,2021-11-22 08:00:00,https://sputniknews.com/20211122/scholars-us--...,Scholars : US & EU Using Ukraine Invasion St...,English,Russia,"Third, the positions of the Democratic neo-lib...",37.0,-1.0,0.095,0.905,0.00,0.095
784,36.0,apnews.com,western,2021-10-27 07:00:00,https://apnews.com/9a111a737a6dd320f2812ccbf35...,WADA says Ukraine routinely violated rules for...,English,United States,The investigation also looked into allegations...,10.0,-1.0,0.068,0.802,0.13,-0.062


### Write JSON

In [69]:
# jsonDf = df.copy(deep=True)
# jsonDf['dateSeen'] = jsonDf['dateSeen'].apply(lambda d : d.timestamp())
# del jsonDf['month']
# del jsonDf['sourceCountry']

# with open('visualizations/histograms/sentData.json', 'w', encoding='utf-8') as f:
#     json.dump(jsonDf.to_dict('records'), f, ensure_ascii=False, indent=4)

## Reading scores

## Create Topic Dataframe

In [70]:
# TopicID	Topic Keywords	Topic Summary	Western Count	Western Freq	Russian Count	Russian Freq	MoscowTimes Count	MoscowTimes Freq	Total Count
numTopics = 15
topicIDs = range(-1,numTopics)
topicStatData = []
for topicID in topicIDs:
    topicRec = dict()
    topicRec['TopicID'] = topicID
    topicRec['Topic Keywords'] = ''
    topicRec['Topic Summary'] = ''
    
    wdf = df[df['category']=='western']
    rdf = df[df['category']=='russian']
    mtdf = df[df['category']=='moscowtimes']
    
    thiswdf = wdf[wdf['sentTopicID']==topicID]
    thisrdf = rdf[rdf['sentTopicID']==topicID]
    thismtdf = mtdf[mtdf['sentTopicID']==topicID]
  
    topicRec['Western Count'] = len(wdf[wdf['sentTopicID']==topicID])
    topicRec['Russian Count'] = len(rdf[rdf['sentTopicID']==topicID])
    topicRec['MoscowTimes Count'] = len(mtdf[mtdf['sentTopicID']==topicID])
    topicRec['Total Count'] = len(df[df['sentTopicID']==topicID])
    
    topicRec['Freq in Western'] = topicRec['Western Count'] / len(wdf)
    topicRec['Freq in Russian'] = topicRec['Russian Count'] / len(rdf)
    topicRec['Freq in MoscowTimes'] = topicRec['MoscowTimes Count'] / len(mtdf)
    topicRec['Total Freq'] = len(df[df['sentTopicID']==topicID])/len(df)
    
    topicRec['Western Senti Avg'] = thiswdf['pos'].mean()- thiswdf['neg'].mean()
    topicRec['Russian Senti Avg'] = thisrdf['pos'].mean() - thisrdf['neg'].mean()
    topicRec['MoscowTimes Senti Avg'] = thismtdf['pos'].mean()- thismtdf['neg'].mean()
    topicRec['Average Senti'] = df[df['sentTopicID']==topicID]['pos'].mean() - df[df['sentTopicID']==topicID]['neg'].mean()
    
    topicStatData.append(topicRec)
    
topicStats = pd.DataFrame.from_records(topicStatData)
topicStats.sample(6)

,TopicID,Topic Keywords,Topic Summary,Western Count,Russian Count,MoscowTimes Count,Total Count,Freq in Western,Freq in Russian,Freq in MoscowTimes,Total Freq,Western Senti Avg,Russian Senti Avg,MoscowTimes Senti Avg,Average Senti
6,5,,,777,1318,180,2275,0.035079,0.047125,0.055710,0.041719,-0.019656,-0.015580,NaN,-0.017958
5,4,,,611,585,75,1271,0.027585,0.020917,0.023213,0.023308,-0.015396,0.005340,NaN,-0.011441
14,13,,,1957,3444,408,5809,0.088352,0.123141,0.126277,0.106527,-0.015825,-0.016890,NaN,-0.016224
9,8,,,386,298,47,731,0.017427,0.010655,0.014547,0.013405,-0.000355,0.011044,NaN,0.001820
2,1,,,267,410,75,752,0.012054,0.014660,0.023213,0.013790,-0.041041,-0.015712,NaN,-0.032964
1,0,,,836,797,73,1706,0.037743,0.028497,0.022594,0.031285,-0.004642,-0.012403,NaN,-0.006418


In [50]:
topicStats.to_excel('data/topic_model_data/topics_and_sentiment_8-3.xlsx')

### Topic Timeline Json

In [51]:
from datetime import datetime

In [52]:
%%time

totalNumTopics= 15
timeTuples = [
    ("2021-10-01", "2021-10-15"), 
    ("2021-10-16", "2021-10-31"), 
    ("2021-11-01", "2021-11-15"), 
    ("2021-11-16", "2021-11-30"), 
    ("2021-12-01", "2021-12-15"), 
    ("2021-12-16", "2021-12-31"),    
    ("2022-01-01", "2022-01-15"),    
    ("2022-01-16", "2022-01-31"),
    ("2022-02-01", "2022-02-15"),
    ("2022-02-16", "2022-02-28"),]

records = df.to_dict('records')

biweeklyDict = dict()
recCatCounts = dict()
for rec in records:
    if rec['category'] in ['western','russian']:
        # add rec to timerange dict
        for i, (startStr, endStr) in enumerate(timeTuples):
            startDate = datetime.strptime(startStr, '%Y-%m-%d')
            endDate = datetime.strptime(endStr, '%Y-%m-%d')
            if rec['dateSeen'] >= startDate and rec['dateSeen'] <= endDate:
                biweeklyDict.setdefault(i, dict())
                for topicID in range(-1,totalNumTopics):
                    biweeklyDict[i].setdefault(topicID, [])
                biweeklyDict[i][rec['sentTopicID']].append(rec)
                recCatCounts.setdefault(i, {'western' : 0,'russian' : 0})
                recCatCounts[i][rec['category']]+= 1
jsonForTimeline = []
for i in biweeklyDict:
    row = dict()
    row['date'] = timeTuples[i][0].replace('-','')
    row[f'western_ct_period_{i}'] = recCatCounts[i]['western']
    row[f'russian_ct_period_{i}'] = recCatCounts[i]['russian']
    for topicID in range(-1,totalNumTopics):
        row['russian_freq_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='russian'])/recCatCounts[i]['russian']
        row['total_freq_topic_'+str(topicID)] = len(biweeklyDict[i][topicID])/(recCatCounts[i]['russian']+recCatCounts[i]['western'])
        row['western_freq_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='western'])/recCatCounts[i]['western']
        row['russian_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='russian'])
        row['western_topic_'+str(topicID)] = len([r for r in biweeklyDict[i][topicID] if r['category']=='western'])
        row['total_topic_'+str(topicID)] = len(biweeklyDict[i][topicID])
    jsonForTimeline.append(row)

CPU times: user 5.7 s, sys: 244 ms, total: 5.95 s
Wall time: 7.54 s


In [55]:
import json
with open('visualizations/timeline/freq_data_8-3.json', 'w', encoding='utf-8') as f:
    json.dump(jsonForTimeline, f, ensure_ascii=False, indent=4)

## Co-occurrence analysis

In [4]:
import itertools
import stanza

In [5]:
stanza.download('en')
nlp = stanza.Pipeline('en')

2022-08-04 17:05:33 INFO: Downloading default packages for language: en (English)...
2022-08-04 17:05:34 INFO: File exists: /Users/paigelee/stanza_resources/en/default.zip.
2022-08-04 17:05:37 INFO: Finished downloading models and saved to /Users/paigelee/stanza_resources.
2022-08-04 17:05:37 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2022-08-04 17:05:37 INFO: Use device: cpu
2022-08-04 17:05:37 INFO: Loading: tokenize
2022-08-04 17:05:37 INFO: Loading: pos
2022-08-04 17:05:38 INFO: Loading: lemma
2022-08-04 17:05:38 INFO: Loading: depparse
2022-08-04 17:05:38 INFO: Loading: sentiment
2022-08-04 17:05:38 INFO: Loading: ner
2022-08-04 17:05:39 INFO: Done loading processors!


In [6]:
import nltk
from nltk.corpus import stopwords

engStopwords = set(stopwords.words('english'))

In [86]:
jsonDict = dict()

In [87]:
# %%time
# get cooccurrence connections
thresh = 100
n = 5

totalWordCt = {
    'western' : 0,
    'russian' : 0
}

sid = SentimentIntensityAnalyzer()

allSentRecords = sdf.to_dict('records')


for cat in ['western','russian']:
    networkJson = dict()
    networkJson['nodes'] = []
    networkJson['links'] = []

    curatedNodes = set()
    linkCounter = dict()
    keywordCounter = dict()
    polarityOfNode = dict()

    windowList = []
    records = []
    for d in allSentRecords:
        if d['category'] == cat and d['sentTopicID'] != -1:
            records.append(d)
            
    random.shuffle(records)
    for i, rec in enumerate(records):
        if i % 1000 == 0:
            print(f'{i}/{len(records)} recs parsed by NLP')
        docText = rec['sentText']
        nlpdoc = nlp(docText)

        ss = sid.polarity_scores(docText)
        polarityScore = ss['pos'] - ss['neg']
        
        wordList = [w for w in nlpdoc.iter_words() if w.text.isalpha() and w.text.lower() not in engStopwords]
        for windowIdx in range(len(wordList) // n + len(wordList) % n):
            startIdx = windowIdx
            endIdx = windowIdx+random.choice(list(range(1,n+1)))
            wordWindow = wordList[startIdx:endIdx]
            tokens = []
            for wordToken in wordWindow:
                formattedToken = wordToken.lemma.lower()
                if formattedToken in engStopwords:
                    continue
                tokens.append(formattedToken)
            windowList.append((tokens, polarityScore))
        for t1 in [w.lemma.lower() for w in wordList]:
            keywordCounter.setdefault(t1, 0)
            keywordCounter[t1] += 1
            polarityOfNode.setdefault(t1, [])
            polarityOfNode[t1].append(polarityScore)
            totalWordCt[cat] += 1
        if totalWordCt[cat] >= 300000:
            print(f'stopping the {cat} count at {totalWordCt[cat]}words')
            break
    print('\tdone nlping docs')
    sufficientNodes = set()
    for key in keywordCounter:
        if keywordCounter[key] >= thresh:
            sufficientNodes.add(key)
    print('\tfound sufficient recs')
    polarityOfLinks= dict()
    
    for tokens, polarityScore in windowList:
        s = set(tokens)
        subsets = [tuple(i) for i in itertools.combinations(s, 2)]
        for t1, t2 in subsets:
            if t1 != t2 and t1 in sufficientNodes and t2 in sufficientNodes:
                label = '_'.join(sorted([t1, t2]))
                linkCounter.setdefault(label, 0)
                linkCounter[label] += 1
                polarityOfLinks.setdefault(label, [])
                polarityOfLinks[label].append(polarityScore)
    print('\tgot linked tokens')
    node2id = dict()
    for i, node in enumerate(sufficientNodes):
        networkJson['nodes'].append({
            'id' : node,
            'occurrences' : keywordCounter[node],
            'nodePolarity' : sum(polarityOfNode[node])/len(polarityOfNode[node])
        })
        node2id[node] = i

    for label in linkCounter:
        source, target = label.split('_')
        networkJson['links'].append({
            'source' : node2id[source],
            'target' : node2id[target],
            'value' : linkCounter[label],
            'polarityScore' : sum(polarityOfLinks[label])/len(polarityOfLinks[label])
        })
    
    jsonDict[cat] = networkJson
    print(f'successfully parsed {len(networkJson["nodes"])} nodes and {len(networkJson["links"])} links')
    
    # with open(f'visualizations/adjmatrix/{cat}network{n}foradjmatrix_8-5_.json', 'w', encoding ='ascii') as json_file:
    #     json.dump(networkJson, json_file, ensure_ascii = True)
    # print('\twrote json')

0/12351 recs parsed by NLP
1000/12351 recs parsed by NLP
2000/12351 recs parsed by NLP
3000/12351 recs parsed by NLP
4000/12351 recs parsed by NLP
5000/12351 recs parsed by NLP
6000/12351 recs parsed by NLP
7000/12351 recs parsed by NLP
8000/12351 recs parsed by NLP
9000/12351 recs parsed by NLP
10000/12351 recs parsed by NLP
11000/12351 recs parsed by NLP
12000/12351 recs parsed by NLP
	done nlping docs
	found sufficient recs
	got linked tokens
successfully parsed 374 nodes and 20577 links
	wrote json
0/16394 recs parsed by NLP
1000/16394 recs parsed by NLP
2000/16394 recs parsed by NLP
3000/16394 recs parsed by NLP
4000/16394 recs parsed by NLP
5000/16394 recs parsed by NLP
6000/16394 recs parsed by NLP
7000/16394 recs parsed by NLP
8000/16394 recs parsed by NLP
9000/16394 recs parsed by NLP
10000/16394 recs parsed by NLP
11000/16394 recs parsed by NLP
12000/16394 recs parsed by NLP
13000/16394 recs parsed by NLP
14000/16394 recs parsed by NLP
15000/16394 recs parsed by NLP
16000/163

In [54]:
with open(f'visualizations/ngramsNetwork/{cat}ngramNet{n}_8-5_w_nodeandlinkpolarity.json', 'w', encoding ='ascii') as json_file:
    json.dump(networkJson, json_file, ensure_ascii = True)
print('\twrote json')

	wrote json


In [53]:
totalWordCt

{'western': 153791, 'russian': 0}

In [63]:
# with open(f'visualizations/ngramsNetwork/{cat}ngramNet{n}_8-4.json', 'w', encoding ='ascii') as json_file:
#     json.dump(networkJson, json_file, ensure_ascii = True)

In [40]:
newJsonDict = jsonDict.copy()

In [15]:
for cat in jsonDict:
    ct = 0
    words = 0
    for d in allSentRecords:
        if d['category'] == cat and d['sentTopicID'] != -1:
            ct += 1
            words += len(word_tokenize(d['sentText']))
    print(cat,'has',ct,'recs and',words,'words')

western has 12351 recs and 367218 words
russian has 16394 recs and 505646 words


In [33]:
ID2node = {
    'western' : dict(),
    'russian' : dict()
}
for cat in jsonDict:
    for i, n in enumerate(jsonDict[cat]['nodes']):
        ID2node[cat][i] = n['id']
node2ID = {
    'russian' : node2id,
    'western' : dict()
}
for i, node in enumerate(jsonDict['western']['nodes']):
    node2ID['western'][node['id']]=i

1.3769640921741308

In [32]:
# how often plan & ukraine or situation & ukraine
for cat in jsonDict:
    for link in jsonDict[cat]['links']:
        for word in ['plan','situation','tension']:
            if set([link['source'],link['target']]) == set([node2ID[cat]['ukraine'],node2ID[cat][word]]):
                print(cat, link, 'ukraine',word)

western {'source': 1456, 'target': 2503, 'value': 50, 'polarityScore': -0.1241} ukraine tension
western {'source': 2180, 'target': 2503, 'value': 65, 'polarityScore': -0.1758615384615385} ukraine plan
western {'source': 3189, 'target': 2503, 'value': 19, 'polarityScore': 0.048736842105263155} ukraine situation
russian {'source': 1593, 'target': 2709, 'value': 107, 'polarityScore': -0.08937383177570091} ukraine tension
russian {'source': 2360, 'target': 2709, 'value': 86, 'polarityScore': -0.052697674418604686} ukraine plan
russian {'source': 3468, 'target': 2709, 'value': 122, 'polarityScore': -0.0074590163934426254} ukraine situation


In [37]:
# highest polarity score
for cat in jsonDict:
    maxPol = 0
    maxLink = ''
    for link in jsonDict[cat]['links']:
        if link['polarityScore'] > maxPol:
            maxPol = link['polarityScore']
            maxLink = link
    print(cat, maxLink,maxPol)

western {'source': 2686, 'target': 2730, 'value': 1, 'polarityScore': 0.688} 0.688
russian {'source': 3564, 'target': 2194, 'value': 1, 'polarityScore': 0.515} 0.515


In [38]:
ID2node['western'][2686],ID2node['western'][2730]

('good', 'like')

In [39]:
ID2node['russian'][3564],ID2node['western'][2194]

('peace', 'prominent')

### Article

In [11]:
# df = pd.read_excel('data/topic model data/topicArticleFrame7-31.xlsx',index_col=0)
# df.sample(5)

FileNotFoundError: [Errno 2] No such file or directory: 'data/topic model data/topicArticleFrame7-31.xlsx'

In [ ]:
# TopicID	Topic Keywords	Topic Summary	Western Count	Western Freq	Russian Count	Russian Freq	MoscowTimes Count	MoscowTimes Freq	Total Count
topicCols = [col for col in df.columns if 'topic' in col]
topicStatData = []
for topicCol in topicCols:
    topicRec = dict()
    topicRec['TopicID'] = int(topicCol.split('_')[1])
    topicRec['Topic Keywords'] = ''
    topicRec['Topic Summary'] = ''
    
    topicRec['Western Count'] = df[df['category']=='western'][topicCol].sum()
    topicRec['Freq in Western'] = df[df['category']=='western'][topicCol].sum()/df[df['category']=='western']['sentsInArticle'].sum()
    
    topicRec['Russian Count'] = df[df['category']=='russian'][topicCol].sum()
    topicRec['Freq in Russian'] = df[df['category']=='russian'][topicCol].sum()/df[df['category']=='russian']['sentsInArticle'].sum()
    
    topicRec['MoscowTimes Count'] = df[df['category']=='moscowtimes'][topicCol].sum()
    topicRec['Freq in MoscowTimes'] = df[df['category']=='moscowtimes'][topicCol].sum()/df[df['category']=='moscowtimes']['sentsInArticle'].sum()
    
    topicRec['Total Count'] = df[topicCol].sum()
    topicRec['Total Freq'] = df[topicCol].sum()/df['sentsInArticle'].sum()
    topicStatData.append(topicRec)
    
topicStats = pd.DataFrame.from_records(topicStatData)
topicStats.sample(6)

,TopicID,Topic Keywords,Topic Summary,Western Count,Freq in Western,Russian Count,Freq in Russian,MoscowTimes Count,Freq in MoscowTimes,Total Count,Total Freq
20,19,,,469,0.018757,674,0.022437,22,0.006681,1165,0.019971
13,12,,,792,0.031675,1289,0.042911,148,0.044944,2229,0.038210
11,10,,,138,0.005519,629,0.020939,56,0.017006,823,0.014108
15,14,,,268,0.010718,353,0.011751,148,0.044944,769,0.013182
18,17,,,842,0.033675,699,0.023270,90,0.027331,1631,0.027959
17,16,,,375,0.014998,79,0.002630,49,0.014880,503,0.008622
